# Feature Engineering

In [190]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import random

In [191]:
# Adjust settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
%matplotlib inline

In [192]:
# Load preprocessed dataset

data_path = '../data/interim/'
df = pd.read_pickle(os.path.join(data_path, 'explored.pkl'))

In [193]:
df.shape

(11703, 22)

## Routing

In [194]:
# Create new feature for routing as dep arpt_arr arpt
df['routing'] = df['flt_dep_airpt'].astype(str) + '_' + df['flt_arr_airpt'].astype(str)
# Drop departure and arrival airport as all information is encoded in routing variable. Furthermore drop leg, as it is only a unique
# identifier similiar to the index.
df.drop(columns=['flt_dep_airpt', 'flt_arr_airpt', 'flt_leg'], inplace=True)

## Crew and Crew Changes

In [195]:
# Save flight crew, cabin crew names and their size in new features and drop the original variable afterwards
cp_crew = []
ca_crew = []
cp_count = []
ca_count = []

for crew in df['flt_TLC_trans']:
    cp_help = []
    ca_help = []
    for member in crew:
        if 'cp' in member:
            cp_help.append(member.partition('_')[0])
        elif 'ca' in member:
            ca_help.append(member.partition('_')[0])
    cp_crew.append(sorted(cp_help))
    ca_crew.append(sorted(ca_help))
    cp_count.append(len(cp_help))
    ca_count.append(len(ca_help))

df['cp_crew'] = cp_crew
df['ca_crew'] = ca_crew
df['cp_count'] = cp_count
df['ca_count'] = ca_count

df = df.drop(columns='flt_TLC_trans')

In [196]:
# Create two new features from flt_crew_change:
# 1. Indicator whether cabin, flight or both crews changed aircraft
# 2. Number of crew members who changed aircraft
def crew_change_cp_ca (row):
    if all(x in row['flt_crewt_change'] for x in ['ca', 'cp']):
        return 'both'
    if 'cp' in row['flt_crewt_change']:
        return 'flight_crew'
    elif 'ca' in row['flt_crewt_change']:
        return 'cabin_crew'
    else:
        return 'None'

def crew_members (row):
    return len(row['flt_crewt_change'])

df['cc_cp_ca'] = df.apply(crew_change_cp_ca, axis=1)
df['cc_count'] = df.apply(crew_members, axis=1)

## Day of the week and hour of the day features

In [197]:
# Create new features for day of the week and hour of the day
df['day_of_week'] = df['flt_sched_dep'].dt.dayofweek
df['hour_of_day_dep'] = df['flt_sched_dep'].dt.hour
df['hour_of_day_arr'] = df['flt_sched_arr'].dt.hour

## Flight of the day feature

In [198]:
# Create a counter for a specific aircraft that indicates which nth flight of the day the considered leg is.

df['dep_date'] = df['flt_sched_dep'].dt.date
df['flight_of_day'] = df.groupby(['dep_date', 'flt_ac_reg']).cumcount()+1
df.drop(columns='dep_date', inplace=True)

## Data Checks: Crew Changing variables

In [199]:
# Check whether flt_crewt_change and flt_crewg match
pd.crosstab(df['flt_crewg'], df['cc_count'])

cc_count,0,1,2,3,4,5,6,8
flt_crewg,,,,,,,,
A,5028,164,121,171,176,185,3,1
B,483,0,0,0,0,0,0,0
B2,0,0,0,55,59,292,15,0
C,0,316,182,96,95,41,1,0
Start,4219,0,0,0,0,0,0,0


There seems to be a mismatch between the variables flt_crewg and the number of crew members who changed the aircraft based on the definition of the variable flt_crewg according to the documentation:
Crew Group: assignment of what happens to the whole crew after a flight
-  Start : First flight of day
-  A : all crew members stay on the aircraft for the next flight
-  B, B2: all crew members switch aircraft for the next flight
-  C: at least one crew member switches aircraft for the next flight

For flt_crewg == A there should only be cc_count == 0 and for flt_crewg == B there should only be cc_count != 0. Therefore a further analysis is done on that.

In [200]:
# Identifying a specific aircraft wherer flt_crewg == A and cc_count != 0 to analyse the crew changes for a specific date
condition = ((df['flt_crewg'] == 'A') & (df['cc_count'] != 0))
sample = df[condition].sample(n=3, random_state=42)

sample

,flt_offblock,flt_onblock,flt_ac_reg,dep_delay,flt_tt,flt_sched_tt,flt_crewg,flt_crewt_change,flt_sched_dep,flt_sched_arr,gnd_mingt,arr_delay,AC Type,routing,sched_gt,act_gt,sched_gt_before,act_gt_before,cp_crew,ca_crew,cp_count,ca_count,cc_cp_ca,cc_count,day_of_week,hour_of_day_dep,hour_of_day_arr,flight_of_day
7827,2019-06-20 17:04:00,2019-06-20 18:59:00,ECLWFX,89.0,64.0,60.0,A,[cp],2019-06-20 15:35:00,2019-06-20 17:10:00,40.0,109.0,E95,East Carmen_Youngland,40.0,-66.0,60.0,59.0,"[Juan Massey, Steve Johnston]","[Melanie Fuentes, Susan Franco, Tammy Mullins]",2,3,flight_crew,1,3,15,17,6
9747,2019-06-25 15:41:00,2019-06-25 17:02:00,ECLBRX,31.0,32.0,30.0,A,"[ca, ca, ca, cp, cp]",2019-06-25 15:10:00,2019-06-25 16:40:00,40.0,22.0,320,East Carmen_Lake Lawrencechester,70.0,49.0,30.0,-13.0,"[Jennifer Adams DDS, Randall Johnson]","[David Giles, Luis Robertson, Steven Blair, Za...",2,4,both,5,1,15,16,5
2040,2019-06-06 07:52:00,2019-06-06 09:07:00,ECLBKX,62.0,41.0,45.0,A,"[ca, ca, cp, cp, ca]",2019-06-06 06:50:00,2019-06-06 08:10:00,40.0,57.0,320,East Carmen_Joneshaven,45.0,50.0,45.0,83.0,"[Jill Conner, Shannon Schmidt]","[Charles Patterson, Dr. Robert White, Jose Tru...",2,4,both,5,3,6,8,2


In [201]:
# Now looking at the crew change history of the first registration for the specific day
df[(df['flt_ac_reg'] == 'ECLWFX') & (df['flt_sched_dep'].dt.strftime('%Y-%m-%d') == '2019-06-20')].iloc[:,[0, 6, 7, 18, 19, 20, 21, 22, 23]]

,flt_offblock,flt_crewg,flt_crewt_change,cp_crew,ca_crew,cp_count,ca_count,cc_cp_ca,cc_count
7561,2019-06-20 04:49:00,Start,[],"[Bryan Murphy, Mrs. Amanda Perez MD]","[Evelyn Russell, Faith Dixon, Olivia Wagner]",2,3,None,0
7617,2019-06-20 06:38:00,A,"[cp, cp]","[Bryan Murphy, Mrs. Amanda Perez MD]","[Andrew Diaz, Julia Jimenez, Laurie Meyer]",2,3,flight_crew,2
7675,2019-06-20 08:35:00,A,[],"[Bryan Murphy, Mrs. Amanda Perez MD]","[Andrew Diaz, Julia Jimenez, Laurie Meyer]",2,3,None,0
7718,2019-06-20 11:54:00,Start,[],"[Juan Massey, Megan Perez]","[Joshua Lindsey, Michael Peters, Teresa Kirk]",2,3,None,0
7771,2019-06-20 14:39:00,A,[],"[Juan Massey, Megan Perez]","[Joshua Lindsey, Michael Peters, Teresa Kirk]",2,3,None,0
7827,2019-06-20 17:04:00,A,[cp],"[Juan Massey, Steve Johnston]","[Melanie Fuentes, Susan Franco, Tammy Mullins]",2,3,flight_crew,1
7863,2019-06-20 17:53:00,A,[],"[Juan Massey, Steve Johnston]","[Melanie Fuentes, Susan Franco, Tammy Mullins]",2,3,None,0


Based on the hypothesis that the names of the crews are being recorded correctly as this might not only be relevant for payroll accounting, but also from regulative perspective the flt_crewg and flt_crewt_change values should be the following:
|   | flt_crewg | flt_crewt_change |
| --- | --- | --- |
7561 | Start | [] |
7617 | C | [ca, ca, ca] |
7675 | A | [] |
7718 | B | [cp, cp, ca, ca, ca] |
7771 | A | [] |
7827 | B | [cp, cp, ca, ca, ca] |
7863 | A | [] |

In [202]:
# Let's have a detailed look at the second example
df[(df['flt_ac_reg'] == 'ECLBRX') & (df['flt_sched_dep'].dt.strftime('%Y-%m-%d') == '2019-06-25')].iloc[:,[0, 6, 7, 18, 19, 20, 21, 22, 23]]

,flt_offblock,flt_crewg,flt_crewt_change,cp_crew,ca_crew,cp_count,ca_count,cc_cp_ca,cc_count
9524,2019-06-25 05:39:00,Start,[],"[Cody Brown, Karen Cruz]","[Jessica Edwards, Joseph Clark, Julie Carey, N...",2,4,None,0
9588,2019-06-25 08:39:00,A,[],"[Cody Brown, Karen Cruz]","[Jessica Edwards, Joseph Clark, Julie Carey, N...",2,4,None,0
9647,2019-06-25 11:34:00,C,"[cp, cp]","[Cody Spencer, Paula Andersen]","[David Giles, Luis Robertson, Shelby Kaufman, ...",2,4,flight_crew,2
9711,2019-06-25 14:26:00,A,"[ca, ca, ca, ca]","[Jennifer Adams DDS, Randall Johnson]","[David Giles, Luis Robertson, Shelby Kaufman, ...",2,4,cabin_crew,4
9747,2019-06-25 15:41:00,A,"[ca, ca, ca, cp, cp]","[Jennifer Adams DDS, Randall Johnson]","[David Giles, Luis Robertson, Steven Blair, Za...",2,4,both,5
9816,2019-06-25 17:51:00,A,[],"[Jennifer Adams DDS, Randall Johnson]","[David Giles, Luis Robertson, Steven Blair, Za...",2,4,None,0


For the second example the values should be the following:
|   | flt_crewg | flt_crewt_change |
| --- | --- | --- |
9524 | Start | [] |
9588 | A | [] |
9647 | B | [cp, cp, ca, ca, ca, ca] |
9711 | C | [cp, cp] |
9747 | C | [ca, ca] |
9816 | A | [] |

In [203]:
# Let's have a detailed look at the third example
df[(df['flt_ac_reg'] == 'ECLBKX') & (df['flt_sched_dep'].dt.strftime('%Y-%m-%d') == '2019-06-06')].iloc[:,[0, 6, 7, 18, 19, 20, 21, 22, 23]]

,flt_offblock,flt_crewg,flt_crewt_change,cp_crew,ca_crew,cp_count,ca_count,cc_cp_ca,cc_count
1972,2019-06-06 04:39:00,Start,[],"[Jill Conner, Shannon Schmidt]","[Charles Patterson, Dr. Robert White, Jose Tru...",2,4,None,0
2040,2019-06-06 07:52:00,A,"[ca, ca, cp, cp, ca]","[Jill Conner, Shannon Schmidt]","[Charles Patterson, Dr. Robert White, Jose Tru...",2,4,both,5
2104,2019-06-06 09:57:00,A,[],"[Jill Conner, Shannon Schmidt]","[Charles Patterson, Dr. Robert White, Jose Tru...",2,4,None,0
2156,2019-06-06 12:27:00,C,"[ca, ca, ca]","[Jill Salinas, Sean Powell]","[Aaron Keller, Candice Smith, Haley Shaw, Patr...",2,4,cabin_crew,3
2198,2019-06-06 13:07:00,A,[ca],"[Laurie Young, Paul Scott]","[Haley Shaw, Jason Randall, Jonathan Pierce, R...",2,4,cabin_crew,1
2232,2019-06-06 16:20:00,A,"[ca, ca, ca, cp]","[Paul Scott, Tristan White]","[Jason Randall, Jonathan Pierce, Rita Bowman, ...",2,4,both,4
2296,2019-06-06 17:50:00,A,[],"[Paul Scott, Tristan White]","[Jason Randall, Jonathan Pierce, Rita Bowman, ...",2,4,None,0
2356,2019-06-06 20:48:00,Start,[],"[Jennifer Patel, Monique Hall]","[Daniel Harrison, Gabriel Mcdaniel, Tammy Simp...",2,4,None,0


For the second example the values should be the following:
|   | flt_crewg | flt_crewt_change |
| --- | --- | --- |
1972 | Start | [] |
2040 | A | [] |
2104 | A | [] |
2156 | B | [cp, cp, ca, ca, ca, ca] |
2198 | B | [cp, cp, ca, ca, ca, ca] |
2232 | C | [cp, ca] |
2296 | A | [] |
2359 | B | [cp, cp, ca, ca, ca, ca] |

Within this sample, only a few entries in flt_crewg and flt_crewt_change match with the crew data over the rotation of a single registration for a day. Furthermore, there is a general error in the variable flt_crewg as it always shows Start for the first leg of the day, however the variable shall state "what happens to the whole crew after a flight". As for the prediction of the arrival delay it is more interesting which crew changes happened prior the flight, the definition of the varibale will be changes to "what happened to the crew after the previous fligth" and therefore Start for the first leg of the day is correct.
Due to this, new features to capture crew changes will be established based on the columns cp_crew and ca_crew for the rotation of a specific aircraft on any given day.

In [204]:
# Creating new features to track crew changes from the flight before: cc_type {0: Start - First flight of the day, 1: No crew change;
# 2: At least one member, but not all of the crew changed; 3: Whole crew changed}; cc_roles [list of elements either ca or cp with an
# entry for every crew member that changed aircraft]

def crew_changes (registrations):
    # Create an empty df that will be appended in every step and finally returned by the function
    cc = pd.DataFrame(columns=['cc_types', 'cc_roles'])
    # Loop through the aircrafts and sort their flights by sched_dep
    for aircraft in registrations:
        indexes = df[df['flt_ac_reg'] == aircraft].sort_values(by='flt_sched_dep').index
        date = '2019-01-01'
        # Loop through the indexes of a specific aircraft
        for idx in indexes:
            cc_roles = []
            # If index has new date and therefore is first flight of the day
            if date not in str(df['flt_sched_dep'].loc[idx]):
                date = str(df['flt_sched_dep'].loc[idx]).partition(' ')[0]
                cp = df['cp_crew'].loc[idx]
                ca = df['ca_crew'].loc[idx]
                help = pd.DataFrame([[0, []]], columns=['cc_types', 'cc_roles'], index=[idx])
            else:
                # Check differences between flight and cabin crew from previous flight
                dif_cp = list(set(cp) - set(df['cp_crew'].loc[idx]))
                dif_ca = list(set(ca) - set(df['ca_crew'].loc[idx]))
                # If no crew members changed
                if (len(dif_cp) == 0) and (len(dif_ca) == 0):
                    help = pd.DataFrame([[1, []]], columns=['cc_types', 'cc_roles'], index=[idx])
                # If all crew members changed
                elif (len(dif_cp) >= len(cp)) and (len(dif_ca) >= len(ca)):
                    for item in dif_cp:
                        cc_roles.append('cp')
                    for item in dif_ca:
                        cc_roles.append('ca')
                    help = pd.DataFrame([[3, cc_roles]], columns=['cc_types', 'cc_roles'], index=[idx])
                else:
                    for item in dif_cp:
                        cc_roles.append('cp')
                    for item in dif_ca:
                        cc_roles.append('ca')
                    help = pd.DataFrame([[2, cc_roles]], columns=['cc_types', 'cc_roles'], index=[idx])
                cp = df['cp_crew'].loc[idx]
                ca = df['ca_crew'].loc[idx]
            cc = cc.append(help)
    return cc

# Create a list of all registrations in the dataset
registrations = df['flt_ac_reg'].unique()

df = pd.merge(df, crew_changes(registrations), left_index=True, right_index=True)
# Finally drop old columns
df.drop(columns=['flt_crewg', 'flt_crewt_change'], inplace=True)

In [205]:
# Create two new features from flt_crew_change:
# 1. Indicator whether cabin, flight or both crews changed aircraft
# 2. Number of crew members who changed aircraft
def crew_change_cp_ca (row):
    if all(x in row['cc_roles'] for x in ['ca', 'cp']):
        return 'both'
    if 'cp' in row['cc_roles']:
        return 'flight_crew'
    elif 'ca' in row['cc_roles']:
        return 'cabin_crew'
    else:
        return 'none'

def crew_members (row):
    return len(row['cc_roles'])

df['cc_cp_ca'] = df.apply(crew_change_cp_ca, axis=1)
df['cc_count'] = df.apply(crew_members, axis=1)
df.drop(columns=['cp_crew', 'ca_crew'], inplace=True)

## Adjusting Data Types prior to further analysis

In [206]:
# Checking current data types
df.dtypes

flt_offblock       datetime64[ns]
flt_onblock        datetime64[ns]
flt_ac_reg                 object
dep_delay                 float64
flt_tt                    float64
flt_sched_tt              float64
flt_sched_dep      datetime64[ns]
flt_sched_arr      datetime64[ns]
gnd_mingt                 float64
arr_delay                 float64
AC Type                  category
routing                    object
sched_gt                  float64
act_gt                    float64
sched_gt_before           float64
act_gt_before             float64
cp_count                    int64
ca_count                    int64
cc_cp_ca                   object
cc_count                    int64
day_of_week                 int64
hour_of_day_dep             int64
hour_of_day_arr             int64
flight_of_day               int64
cc_types                   object
cc_roles                   object
dtype: object

In [207]:
convert_dict = {'gnd_mingt': 'object',
                'cp_count': 'object',
                'ca_count': 'object',
                'cc_count': 'object',
                'day_of_week': 'object',
                'hour_of_day_dep': 'object',
                'hour_of_day_arr': 'object',
                'flight_of_day': 'object'
}
df = df.astype(convert_dict)

In [208]:
df.shape

(11703, 26)

In [209]:
df.to_csv('../data/interim/features.csv')
df.to_pickle('../data/interim/features.pkl')